In [1]:
# Inspired by https://www.sciencedirect.com/science/article/pii/S0377221723000735
# https://www.sciencedirect.com/science/article/pii/S0167637704000537
# https://medium.com/swlh/techniques-for-subtour-elimination-in-traveling-salesman-problem-theory-and-implementation-in-71942e0baf0c

In [2]:
import gurobipy as gp
import math
import pandas as pd

In [3]:
# Read data

# Read files
file = "Information Scrapyard challenge.xlsx"
production_plan = pd.read_excel(file, sheet_name = "Production plan")

grade_mix = pd.read_excel(file, sheet_name = "Grade mix large buckets")
grade_mix.columns.values[0] = "Grade"

scrap_types = pd.read_excel(file, sheet_name = "Scraptype")
scrap_types.columns.values[0] = "Scraptype"
scrap_types.columns.values[9] = "Capacity"
# print(f"{scrap_types}")

# Store density order and capacity of each scrap type
scrap_density = {}
scrap_storage = {}
for index, row in scrap_types.iloc[0:32,:].iterrows():
    scrap_density[row["Scraptype"]] = row["Layer bucket"]
    scrap_storage[row["Scraptype"]] = row["Capacity"]

set_all_scraps = range(0, 32)

#print(f"{scrap_density}")
#print(f"{scrap_storage}")

# Create a set of all grades produced and a list of the quantities produced
all_grades_produced = production_plan.query("Installation=='EAF1' | Installation=='EAF2'")["Grade"]
grades_produced = list(set(all_grades_produced))
grades_count_dict = all_grades_produced.value_counts().to_dict()
all_grades_produced = [grade.removesuffix("_EAF") for grade in all_grades_produced]
grades_produced = [grade.removesuffix("_EAF") for grade in grades_produced]

# Create a dictionary which holds the recipe for each grade
grade_recipe_dict = {}
for grade in grades_produced:
    row = grade_mix.loc[grade_mix["Grade"] == grade].iloc[:,2:].values.tolist()[0]
    grade_recipe_dict[grade] = row

# List all unused scrap types
scrap_types_unused = list(range(0, 32))
for row in grade_recipe_dict.values():
    for i, value in enumerate(row):
        if value > 0 and i in scrap_types_unused:
            scrap_types_unused.remove(i)
print(f"{scrap_types_unused}")

# Create dict to map new scrap indices to old scrap indices
new_to_old = {}
new_index = 0
for old_scrap in set_all_scraps:
    if old_scrap in scrap_types_unused:
        continue
    new_to_old[new_index] = old_scrap
    new_index += 1
print(f"{new_to_old}")

# Remove unused scrap types from rows in dictionary
for key, value in grade_recipe_dict.items():
    new = [value[i] for i in set_all_scraps if i not in scrap_types_unused]
    grade_recipe_dict[key] = new
print(f"{grade_recipe_dict}")

# Transform the recipes into a list of scrap types used
recipes = []
recipes_quantities = []
for r_i, tons in grade_recipe_dict.items():
    recipe = []
    for i, value in enumerate(tons):
        if value > 0:
            recipe.append(i)
    recipes.append(recipe)
    recipes_quantities.append(grades_count_dict[r_i + "_EAF"])
print(f"{recipes_quantities}")

# Store the scrap types that must precede each scrap type in a recipe
recipes_preceders = []
for recipe in recipes:
    preceders = {}
    for scrap in recipe:
        preceders[scrap] = [s for s in recipe if scrap_density[new_to_old[s]] < scrap_density[new_to_old[scrap]]]
    recipes_preceders.append(preceders)

#recipes_preceders[0]

scrap_capacity = [scrap_storage[s] for s in range(0, 32) if s not in scrap_types_unused]

# Set number of scraps used
scraps = 32 - len(scrap_types_unused)

# Set maximum number of boxes
max_boxes = 39
max_points = max_boxes * 2 + 2

yard_length = 505
yard_sides = 2 # 0 for left, 1 for right
min_box_length = 10
wall_length = 1.7
box_width = 40
box_height = 10

set_scraps = range(0, scraps)
set_boxes = range(0, max_boxes)
set_points = range(0, max_points)
set_recipes = range(0, len(recipes))
set_yard_sides = range(0, yard_sides)

[2, 10, 11, 26, 30, 31]
{0: 0, 1: 1, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 12, 10: 13, 11: 14, 12: 15, 13: 16, 14: 17, 15: 18, 16: 19, 17: 20, 18: 21, 19: 22, 20: 23, 21: 24, 22: 25, 23: 27, 24: 28, 25: 29}
{'K400': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 73.8, 0.0, 0.0, 0.0, 17.6, 41.6, 10.4, 3.0, 4.0, 15.0, 0.0, 0.0, 0.0, 0.0], 'LK60': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 17.5, 0.0, 0.0, 0.0, 0.0, 0.0, 97.8, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0, 5.0], 'L620': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 63.90000000000001, 0.0, 0.0, 43.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 12.0, 0.0, 0.0, 0.0], 'L650': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 88.8, 0.0, 0.0, 41.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0], 'K320': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 49.4, 0.0, 0.0, 0.0, 13.0, 18.3, 49.8, 11.0, 0.0, 5.6, 0.0, 14.5, 4.0, 0.0, 13.1, 0.0, 4.0, 5.0, 0.0], 'K330': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [4]:
# # Override parameters for testing
# scraps = 6 #32
# max_boxes = 6 #39
# max_points = max_boxes * 2 + 2

# yard_length = 505
# yard_sides = 2
# min_box_length = 10
# wall_length = 1.7
# box_width = 20
# box_height = 10

# set_scraps = range(0, scraps)
# set_boxes = range(0, max_boxes)
# set_points = range(0, max_points)
# set_yard_sides = range(0, yard_sides)

# scrap_capacity = [4000, 2000, 2000, 2000, 2000, 2000]

# # Minium of 3 scraps per recipe, MUST USE ALL SCRAPS IN AT LEAST ONE RECIPE
# recipes = [[0, 2, 1, 3], [1, 2, 3], [3, 2, 4, 5], [4, 3, 2]]
# recipes_quantities = [1, 1, 1, 1]
# recipes_strict_order_i = [[0, 3], [0, 1, 2], [0, 1, 2], [0, 1, 2]]
# recipes_preceders = []
# for r_i, recipe in enumerate(recipes):
#     preceders_dict = {}
#     for s_i, scrap in enumerate(recipe):
#         if s_i in recipes_strict_order_i[r_i]:
#             preceders_dict[scrap] = recipe[:s_i]
#         else:
#             preceders_dict[scrap] = [recipe[i] for i in range(0, len(recipe)) if i in recipes_strict_order_i[r_i] and i < s_i]
#     recipes_preceders.append(preceders_dict)
       
# set_recipes = range(0, len(recipes))
# recipes_preceders

In [5]:
# Create a new model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02


In [6]:
# Create variables

# Scrap s is assigned to box b on side lr
x = {}
for lr in set_yard_sides:
    for scrap in set_scraps:
        for box in set_boxes:
            x[scrap, box, lr] = m.addVar(vtype='B', name=f"x[{scrap},{box},{lr}]")

# Scrap s is assigned to point w on side lr
a = {}
for lr in set_yard_sides:
    for scrap in set_scraps:
        for point in set_points[1:]:
            a[scrap, point, lr] = m.addVar(vtype='B', name=f"a[{scrap},{point},{lr}]")

# Box b on side lr is in solution path for recipe r
y = {}
for lr in set_yard_sides:
    for box in set_boxes:
        for recipe in set_recipes:
            y[box, lr, recipe] = m.addVar(vtype='B', name=f"y[{box},{recipe},{lr}]")

# Box b on side lr is in the global solution
g = {}
for lr in set_yard_sides:
    for box in set_boxes:
        g[box, lr] = m.addVar(vtype='B', name=f"g[{box},{lr}]")

# Distance between start and point w on side lr
w = {}
for lr in set_yard_sides:
    for w1 in set_points:
        w[w1, lr] = m.addVar(vtype='C', lb=0, ub=yard_length + 100, name=f"w[{w1},{lr}]")

# Distance between two points w1 and w2
d = {}
for lr1 in set_yard_sides:
    for lr2 in set_yard_sides:
        for w1 in set_points:
            for w2 in set_points:
                if lr1 == lr2 and w1 == w2: continue
                d[w1, lr1, w2, lr2] = m.addVar(vtype='C', lb=-yard_length + 100, ub=yard_length + 100, name=f"d[{w1},{lr1},{w2},{lr2}]")

# Absolute distance between two points w1 and w2
d_abs = {}
for lr1 in set_yard_sides:
    for lr2 in set_yard_sides:
        for w1 in set_points:
            for w2 in set_points:
                if lr1 == lr2 and w1 == w2: continue
                d_abs[w1, lr1, w2, lr2] = m.addVar(vtype='C', lb=0, ub=yard_length + 100, name=f"d_abs[{w1},{lr1},{w2},{lr2}]")

# Length of box b on side lr
l = {}
for lr in set_yard_sides:
    for box in set_boxes:
        l[box, lr] = m.addVar(vtype='C', lb=0, ub=yard_length, name=f"l[{box},{lr}]")

# Point w on side lr is in solution path for recipe r
p = {}
for lr in set_yard_sides:
    for w1 in set_points:
        for recipe in set_recipes:
            p[w1, lr, recipe] = m.addVar(vtype='B', name=f"p[{w1},{lr},{recipe}]")

# Edge w1 -> w2 is in solution path for recipe r
z = {}
for lr1 in set_yard_sides:
    for lr2 in set_yard_sides:
        for w1 in set_points:
            for w2 in set_points:
                if lr1 == lr2 and w1 == w2: continue
                for recipe in set_recipes:
                    z[w1, lr1, w2, lr2, recipe] = m.addVar(vtype='B', name=f"z[{w1},{lr1},{w2},{lr2},{recipe}]")

# Capacity of box b on side lr
c = {}
for lr in set_yard_sides:
    for box in set_boxes:
        c[box, lr] = m.addVar(vtype='C', lb=0, name=f"c[{box},{lr}]")

# MTZ value in a point on a side in a recipe
mtz = {}
for recipe in set_recipes:
    for lr in set_yard_sides:
        for w1 in set_points[1:-1]:
            mtz[w1, lr, recipe] = m.addVar(vtype='C', lb=0, name=f"mtz[{w1},{lr},{recipe}]")

# Actual MTZ for scraps in a recipe
mtz_s = {}
for recipe in set_recipes:
    for scrap in set_scraps:
        mtz_s[scrap, recipe] = m.addVar(vtype='C', lb=0, name=f"mtz_s[{scrap},{recipe}]")

# MTZ for a scrap in a point on a side in a recipe
mtz_s_a = {}
for recipe in set_recipes:
    for scrap in set_scraps:
        for lr in set_yard_sides:
            for w1 in set_points[1:-1]:
                mtz_s_a[scrap, w1, lr, recipe] = m.addVar(vtype='C', lb=0, name=f"mtz_s_a[{scrap},{w1},{lr},{recipe}]")

# Distance of path of recipe r (for use in objective function)
r_d = {}
for recipe in set_recipes:
    r_d[recipe] = m.addVar(vtype='C', name=f"r_d[{recipe}]")

# Furthest point in the yard (for use in objective function)
furthest_point = m.addVar(vtype='C', name=f"furthest_point")


In [7]:
# Add constraints

# Each scrap used in a recipe is assigned to at least one box
for s in set_scraps:
    m.addConstr(sum(x[s, b, lr] for b in set_boxes for lr in set_yard_sides) >= 1)

# point 0 is in solution path for all recipes
for r in set_recipes:
    m.addConstr(p[0, 0, r] == 1)
    m.addConstr(p[0, 1, r] == 0)

# Each box is assigned at most one scrap
for lr in set_yard_sides:
    for b in set_boxes:
        m.addConstr(sum(x[s, b, lr] for s in set_scraps) <= g[b, lr])

# The distance to point 1 is 100m
m.addConstr(w[1, 0] == 100)
m.addConstr(w[0, 0] == 0)
m.addConstr(w[1, 1] == 100)
m.addConstr(w[0, 1] == 0)

# Dummy points are not in the solution path for any recipe
for r in set_recipes:
    for lr in set_yard_sides:
        m.addConstr(p[max_points - 1, lr, r] == 0) 

# Capacity of a box is determined by the formula
for lr in set_yard_sides:
    for b in set_boxes:
        m.addConstr(c[b, lr] == box_height * box_width * l[b, lr])

# The total capacity of boxes with a scrap s must be greater than the required volume
for s in set_scraps:
    m.addConstr(sum(c[b, lr] * x[s, b, lr] for b in set_boxes for lr in set_yard_sides) >= scrap_capacity[s])

# Points p1 and p2 of box b are assigned to the same scrap
for lr in set_yard_sides:
    for b in set_boxes:
        for s in set_scraps:
            m.addConstr(x[s, b, lr] == a[s, 2*b + 1, lr])
            m.addConstr(x[s, b, lr] == a[s, 2*b + 2, lr])

# The number of boxes in a solution for recipe r is equal to the number of scraps required by the recipe
for r in set_recipes:
    m.addConstr(sum(y[b, lr, r] for b in set_boxes for lr in set_yard_sides) == len(recipes[r]))

# There is exactly one box b in the solution path for recipe r that corresponds to each scrap s
for r in set_recipes:
    for s in recipes[r]:
        m.addConstr(sum(x[s, b, lr] * y[b, lr, r] for b in set_boxes for lr in set_yard_sides) == 1)

# There is one incoming edge to each point w in the solution path for recipe r
for r in set_recipes:
    for lr2 in set_yard_sides:
        for w2 in set_points:
            m.addConstr(sum(z[w1, lr1, w2, lr2, r] for w1 in set_points for lr1 in set_yard_sides if w1 != w2 or lr1 != lr2) == p[w2, lr2, r])

# There is one outgoing edge from each point w in the solution path for recipe r
for r in set_recipes:
    for lr1 in set_yard_sides:
        for w1 in set_points:
            m.addConstr(sum(z[w1, lr1, w2, lr2, r] for w2 in set_points for lr2 in set_yard_sides if w1 != w2 or lr1 != lr2) == p[w1, lr1, r])

# The length of a box is the distance between the first and last point of the box
for lr in set_yard_sides:
    for b in set_boxes:
        m.addConstr(l[b, lr] == w[2*b + 2, lr] - w[2*b + 1, lr])

# The length of a box in the solution is larger than the minimum
for lr in set_yard_sides:
    for b in set_boxes:
        m.addGenConstrIndicator(g[b, lr], True, l[b, lr] >= min_box_length)

# The length of a box not in the solution is 0
for lr in set_yard_sides:
    for b in set_boxes:
        m.addGenConstrIndicator(g[b, lr], False, l[b, lr] == 0)

# The distance between edge points of a wall (in the solution) is equal to the length of the wall
for lr in set_yard_sides:
    for b in set_boxes:
        m.addConstr(w[2*b + 3, lr] - w[2*b + 2, lr] == wall_length * g[b, lr])
        # m.addGenConstrIndicator(g[b], True, w[2*b + 3] - w[2*b + 2] == 1.7)

# Calculate the distances between all points
for lr1 in set_yard_sides:
    for lr2 in set_yard_sides:
        for w1 in set_points:
            for w2 in set_points:
                if lr1 == lr2 and w1 == w2: continue
                m.addConstr(d[w1, lr1, w2, lr2] == w[w2, lr2] - w[w1, lr1])

# Calculate the absolute distances between all points
for lr1 in set_yard_sides:
    for lr2 in set_yard_sides:
        for w1 in set_points:
            for w2 in set_points:
                if lr1 == lr2 and w1 == w2: continue
                m.addGenConstrAbs(d_abs[w1, lr1, w2, lr2], d[w1, lr1, w2, lr2])

# Box b is in the global solution if it is in the solution path for any recipe
for lr1 in set_yard_sides:
    for b in set_boxes:
        m.addGenConstrIndicator(g[b, lr1], True, sum(y[b, lr2, r] for lr2 in set_yard_sides for r in set_recipes) >= 1)

# Box b is not in the solution path for any recipe if it is not in the global solution
for lr in set_yard_sides:
    for b in set_boxes:
        for r in set_recipes:
            m.addConstr(y[b, lr, r] <= g[b, lr])

# If a box b is not in the global solution, all later boxes are also not in the global solution
for lr1 in set_yard_sides:
    for b1 in set_boxes:
        m.addConstr(sum(g[b2, lr2] for b2 in set_boxes if b2 > b1 for lr2 in set_yard_sides) <= g[b1, lr1] * max_boxes)

# If a box is in the solution path for recipe r, then one of the edge points of the box is also in the solution path for recipe r
for r in set_recipes:
    for lr in set_yard_sides:
        for b in set_boxes:
            m.addConstr(y[b, lr, r] == p[2*b + 1, lr, r] + p[2*b + 2, lr, r])

# Only one edge of a box can be in the solution path for recipe r
for r in set_recipes:
    for lr in set_yard_sides:
        for b in set_boxes:
            m.addConstr(p[2*b + 1, lr, r] + p[2*b + 2, lr, r] <= 1)

# The distance of a recipe is the sum of the distances between the points in the solution path
for r in set_recipes:
    m.addConstr(r_d[r] == sum(z[w1, lr1, w2, lr2, r] * d_abs[w1, lr1, w2, lr2] for w1 in set_points for w2 in set_points for lr1 in set_yard_sides for lr2 in set_yard_sides if w1 != w2 or lr1 != lr2))

# The furthest point in the yard is the maximum distance of all points
m.addGenConstrMax(furthest_point, [w[max_points - 1, lr] for lr in set_yard_sides], name="max_furthest_point")

# Ordering constraints    
# No subtours in recipe solution paths using MTZ values
for r in set_recipes:
    for lr1 in set_yard_sides:
        for lr2 in set_yard_sides:
            for w1 in set_points[1:-1]:
                for w2 in set_points[1:-1]:
                    if lr1 == lr2 and w1 == w2: continue
                    # m.addConstr(mtz[w2, r] >= mtz[w1, r] + 1 - 1000 * (1 - z[w1, w2, r]))
                    m.addGenConstrIndicator(z[w1, lr1, w2, lr2, r], True, mtz[w2, lr2, r] >= mtz[w1, lr1, r] + 1)

# MTZ values for points not in the solution path are 0
for r in set_recipes:
    for lr in set_yard_sides:
        for w1 in set_points[1:-1]:
            m.addGenConstrIndicator(p[w1, lr, r], False, mtz[w1, lr, r] == 0)

# MTZ values for scrap ordering are equal to the MTZ value of a point if the scrap is assigned to the point
for r in set_recipes:
    for s in set_scraps:
        for lr in set_yard_sides:
            for w1 in set_points[1:-1]:
                m.addConstr(mtz_s_a[s, w1, lr, r] == mtz[w1, lr, r] * a[s, w1, lr])

# MTZ values for scrap ordering are equal to the maximum mtz value of the points assigned to the scrap
for r in set_recipes:
    for s in set_scraps:
        m.addGenConstrMax(mtz_s[s, r], [mtz_s_a[s, w1, lr, r] for w1 in set_points[1:-1] for lr in set_yard_sides], name=f"max_mtz_s[{s},{r}]")

# Precendence of scrap types in recipes
for r in set_recipes:
    for s_recipe in recipes[r]:
        for s_precedence in recipes_preceders[r][s_recipe]:
            m.addConstr(mtz_s[s_recipe, r] >= mtz_s[s_precedence, r] + 1)



In [8]:
# Set objective function

# Travel distance
objective_distance = gp.quicksum(r_d[r] * recipes_quantities[r] for r in set_recipes)

# Total length of boxes
# objective = gp.quicksum(l[b, lr] for b in set_boxes for lr in set_yard_sides)

# Length of the yard
objective_yard_length = furthest_point - 100

# m.setObjective(objective, gp.GRB.MINIMIZE)

# Multiple objectives
m.setObjectiveN(objective_distance, 0, 1)
m.setObjectiveN(objective_yard_length, 1, 0)

In [9]:
# Statistics
m.printStats()


Statistics for model Unnamed:
  Linear constraint matrix    : 0 Constrs, 0 Vars, 0 NZs
  Matrix coefficient range    : [ 0, 0 ]
  Objective coefficient range : [ 0, 0 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 0, 0 ]


In [10]:
# Solve
m.Params.timeLimit = 60 * 60
#m.params.cuts = 0
# Focus on finding feasable solutions
# m.params.mipfocus = 1
m.optimize()

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 PRO 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 38104 rows, 476249 columns and 818552 nonzeros
Model fingerprint: 0x283065ff
Model has 56933 quadratic constraints
Model has 366743 general constraints
Variable types: 110543 continuous, 365706 integer (365706 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 1e+02]
  QRHS range       [1e+00, 2e+04]
  GenCon rhs range [1e+00, 1e+01]
  GenCon coe range [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectiv

In [11]:
# Store solution
if m.SolCount > 0:
    m.write("solution_2_sides.sol")

GurobiError: Unable to retrieve attribute 'X'

In [25]:
# # Read solution
# m.update()
# m.read("solution.sol")
# m.Params.timeLimit = 2
# m.optimize()

In [26]:
# Show solution

# Print objective values
print(f"Objective distance: {objective_distance.getValue()}")
print(f"Objective yard length: {objective_yard_length.getValue()}")

# Debug

# Print boxes in solution
boxes_in_solution = [0, 0]
for lr in set_yard_sides:
    for box in set_boxes:
        if g[box, lr].X > 0:
            print(f"Box {box} on side {lr} is in solution")
            boxes_in_solution[lr] += 1

# Print box capacities
for lr in set_yard_sides:
    for box in set_boxes:
        if c[box, lr].X > 0:
            print(f"Box {box} on side {lr} capacity: {c[box, lr].X}")

# Print scrap assignments
scrap_assignments = {}
for lr in set_yard_sides:
    for box in set_boxes:
        for scrap in set_scraps:
            if x[scrap, box, lr].X > 0:
                scrap_assignments[box, lr] = scrap
                print(f"Scrap {scrap} is assigned to box {box} on side {lr}")

# Print points in solution
for lr in set_yard_sides:
    for point in set_points:
        if p[point, lr, 0].X > 0:
            print(f"Point {point} on side {lr} is in solution")

# Print box lengths
for lr in set_yard_sides:
    for box in set_boxes:
        if l[box, lr].X > 0:
            print(f"Box {box} on side {lr} length: {l[box, lr].X}")

# Print solution paths
def get_box_index(point):
    return math.floor((point - 1) // 2)

# Print what should be order of scraps in recipes
for recipe in set_recipes:
    print(f"Recipe {recipe} scrap mtz:")
    for scrap in set_scraps:
        print(f"MTZ scrap {scrap} {mtz_s[scrap, recipe].X}")

for recipe in set_recipes:
    print(f"Recipe {recipe} solution path:")
    for lr1 in set_yard_sides:
        for lr2 in set_yard_sides:
            for w1 in set_points:
                for w2 in set_points:
                    if lr1 == lr2 and w1 == w2: continue
                    if z[w1, lr1, w2, lr2, recipe].X > 0:
                        # print(f" {w1} -> {w2}")
                        box1 = get_box_index(w1)
                        box2 = get_box_index(w2)

                        if w1 == 0:
                            print(f"Start -> Box {box2} Side {lr2}: Scrap: {scrap_assignments[box2, lr2]}")
                        elif w2 == 0:
                            print(f"Box {box1} Side {lr1}: Scrap: {scrap_assignments[box1, lr1]} -> Start")
                        else:
                            print(f"Box {box1} Side {lr1}: Scrap: {scrap_assignments[box1, lr1]} -> Box {box2}: Scrap: {scrap_assignments[box2, lr2]}")
    

Objective distance: 940.3999968688042
Objective yard length: 45.099999999999994
Box 0 on side 0 is in solution
Box 1 on side 0 is in solution
Box 2 on side 0 is in solution
Box 0 on side 1 is in solution
Box 1 on side 1 is in solution
Box 2 on side 1 is in solution
Box 0 on side 0 capacity: 2000.0
Box 1 on side 0 capacity: 2000.0
Box 2 on side 0 capacity: 4000.0
Box 0 on side 1 capacity: 2000.0
Box 1 on side 1 capacity: 2000.0
Box 2 on side 1 capacity: 2000.0
Scrap 2 is assigned to box 0 on side 0
Scrap 4 is assigned to box 1 on side 0
Scrap 0 is assigned to box 2 on side 0
Scrap 3 is assigned to box 0 on side 1
Scrap 1 is assigned to box 1 on side 1
Scrap 5 is assigned to box 2 on side 1
Point 0 on side 0 is in solution
Point 1 on side 0 is in solution
Point 5 on side 0 is in solution
Point 1 on side 1 is in solution
Point 3 on side 1 is in solution
Box 0 on side 0 length: 10.0
Box 1 on side 0 length: 10.0
Box 2 on side 0 length: 20.0
Box 0 on side 1 length: 10.0
Box 1 on side 1 lengt